In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = ShapesConfig()
config.display()

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
class ShapesDataset(utils.Dataset):
    """Generates the shapes synthetic dataset. The dataset consists of simple
    shapes (triangles, squares, circles) placed randomly on a blank surface.
    The images are generated on the fly. No file access required.
    """

    def load_shapes(self, count, height, width):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.add_class("shapes", 1, "square")
        self.add_class("shapes", 2, "circle")
        self.add_class("shapes", 3, "triangle")

        # Add images
        # Generate random specifications of images (i.e. color and
        # list of shapes sizes and locations). This is more compact than
        # actual images. Images are generated on the fly in load_image().
        for i in range(count):
            bg_color, shapes = self.random_image(height, width)
            self.add_image("shapes", image_id=i, path=None,
                           width=width, height=height,
                           bg_color=bg_color, shapes=shapes)

    def load_image(self, image_id):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly from the
        specs in image_info.
        """
        info = self.image_info[image_id]
        bg_color = np.array(info['bg_color']).reshape([1, 1, 3])
        image = np.ones([info['height'], info['width'], 3], dtype=np.uint8)
        image = image * bg_color.astype(np.uint8)
        for shape, color, dims in info['shapes']:
            image = self.draw_shape(image, shape, dims, color)
        return image

    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shapes":
            return info["shapes"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        info = self.image_info[image_id]
        shapes = info['shapes']
        count = len(shapes)
        mask = np.zeros([info['height'], info['width'], count], dtype=np.uint8)
        for i, (shape, _, dims) in enumerate(info['shapes']):
            mask[:, :, i:i+1] = self.draw_shape(mask[:, :, i:i+1].copy(),
                                                shape, dims, 1)
        # Handle occlusions
        occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
        for i in range(count-2, -1, -1):
            mask[:, :, i] = mask[:, :, i] * occlusion
            occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
        # Map class names to class IDs.
        class_ids = np.array([self.class_names.index(s[0]) for s in shapes])
        return mask.astype(np.bool), class_ids.astype(np.int32)

    def draw_shape(self, image, shape, dims, color):
        """Draws a shape from the given specs."""
        # Get the center x, y and the size s
        x, y, s = dims
        if shape == 'square':
            cv2.rectangle(image, (x-s, y-s), (x+s, y+s), color, -1)
        elif shape == "circle":
            cv2.circle(image, (x, y), s, color, -1)
        elif shape == "triangle":
            points = np.array([[(x, y-s),
                                (x-s/math.sin(math.radians(60)), y+s),
                                (x+s/math.sin(math.radians(60)), y+s),
                                ]], dtype=np.int32)
            cv2.fillPoly(image, points, color)
        return image

    def random_shape(self, height, width):
        """Generates specifications of a random shape that lies within
        the given height and width boundaries.
        Returns a tuple of three valus:
        * The shape name (square, circle, ...)
        * Shape color: a tuple of 3 values, RGB.
        * Shape dimensions: A tuple of values that define the shape size
                            and location. Differs per shape type.
        """
        # Shape
        shape = random.choice(["square", "circle", "triangle"])
        # Color
        color = tuple([random.randint(0, 255) for _ in range(3)])
        # Center x, y
        buffer = 20
        y = random.randint(buffer, height - buffer - 1)
        x = random.randint(buffer, width - buffer - 1)
        # Size
        s = random.randint(buffer, height//4)
        return shape, color, (x, y, s)

    def random_image(self, height, width):
        """Creates random specifications of an image with multiple shapes.
        Returns the background color of the image and a list of shape
        specifications that can be used to draw the image.
        """
        # Pick random background color
        bg_color = np.array([random.randint(0, 255) for _ in range(3)])
        # Generate a few random shapes and record their
        # bounding boxes
        shapes = []
        boxes = []
        N = random.randint(1, 4)
        for _ in range(N):
            shape, color, dims = self.random_shape(height, width)
            shapes.append((shape, color, dims))
            x, y, s = dims
            boxes.append([y-s, x-s, y+s, x+s])
        # Apply non-max suppression wit 0.3 threshold to avoid
        # shapes covering each other
        keep_ixs = utils.non_max_suppression(np.array(boxes), np.arange(N), 0.3)
        shapes = [s for i, s in enumerate(shapes) if i in keep_ixs]
        return bg_color, shapes

In [ ]:
# Training dataset
dataset_train = ShapesDataset()
dataset_train.load_shapes(500, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_train.prepare()

# Validation dataset
dataset_val = ShapesDataset()
dataset_val.load_shapes(50, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_val.prepare()

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

In [ ]:
node_def: name: "Placeholder"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: -1
      }
      dim {
        size: -1
      }
      dim {
        size: 3
      }
    }
  }
}


self: FuncGraph(name=conv1_scratch_graph, id=1703516560384)

inputs: []

dtypes: None

control_inputs: []

self._default_original_op: None

op_def: name: "Placeholder"
output_arg {
  name: "output"
  type_attr: "dtype"
}
attr {
  name: "dtype"
  type: "type"
}
attr {
  name: "shape"
  type: "shape"
  default_value {
    shape {
      unknown_rank: true
    }
  }
}

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Input layer
inputs = Input(shape=(10,))

# Hidden layer
x = Dense(32, activation='relu')(inputs)

# Output layer
outputs = Dense(1)(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Custom loss function
def custom_loss(y_true, y_pred):
    # Compute the custom loss
    loss = tf.reduce_mean(tf.square(y_true - y_pred))
    return loss

# Add the custom loss to the model
print(custom_loss(inputs, outputs))


In [10]:
import tensorflow as tf
import numpy as np

# Create a tensor from a Python list
tensor = tf.convert_to_tensor([1, 2, 3])

# Create a tensor from a NumPy array
array = np.array([1, 2, 3])
tensor = tf.convert_to_tensor(array)
tensor

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3])>

In [23]:
y_true = [None, None, None, None, None, None, None, None, None, None, None, None, None, None]
y_pred = ["<tf.Tensor 'mask_rcnn/rpn_class_logits/concat:0' shape=(None, None, 2) dtype=float32>", "<tf.Tensor 'mask_rcnn/rpn_class/concat:0' shape=(None, None, 2) dtype=float32>", "<tf.Tensor 'mask_rcnn/rpn_bbox/concat:0' shape=(None, None, 4) dtype=float32>", "<KerasTensor: shape=(8, None, 4) dtype=float32 (created by layer 'mrcnn_class_logits')>", "<KerasTensor: shape=(8, None, 4) dtype=float32 (created by layer 'mrcnn_class')>", "<KerasTensor: shape=(8, None, 4, 4) dtype=float32 (created by layer 'mrcnn_bbox')>", "<KerasTensor: shape=(8, None, 28, 28, 4) dtype=float32 (created by layer 'mrcnn_mask')>", "<KerasTensor: shape=(8, None, 4) dtype=float32 (created by layer 'ROI')>", "<KerasTensor: shape=(8, None, 4) dtype=float32 (created by layer 'output_rois')>", "<tf.Tensor 'mask_rcnn/rpn_class_loss/cond/Identity:0' shape=() dtype=float32>", "<tf.Tensor 'mask_rcnn/rpn_bbox_loss/cond/Identity:0' shape=() dtype=float32>", "<KerasTensor: shape=() dtype=float32 (created by layer 'mrcnn_class_loss')>", "<KerasTensor: shape=() dtype=float32 (created by layer 'mrcnn_bbox_loss')>", "<KerasTensor: shape=() dtype=float32 (created by layer 'mrcnn_mask_loss')>"]
sample_weight = [None, None, None, None, None, None, None, None, None, None, None, None, None, None]
t_losses = [None, None, None, None, None, None, None, None, None, None, None, None, None, None]
t_loss_weights = [None, None, None, None, None, None, None, None, None, None, None, None, None, None]
t_per_output_metrics = [None, None, None, None, None, None, None, None, None, None, None, None, None, None]
zip_args = (
            y_true,
            y_pred,
            sample_weight,
            t_losses,
            t_loss_weights,
            t_per_output_metrics,
        )

In [24]:
for item in zip(*zip_args):
    print(item)

(None, "<tf.Tensor 'mask_rcnn/rpn_class_logits/concat:0' shape=(None, None, 2) dtype=float32>", None, None, None, None)
(None, "<tf.Tensor 'mask_rcnn/rpn_class/concat:0' shape=(None, None, 2) dtype=float32>", None, None, None, None)
(None, "<tf.Tensor 'mask_rcnn/rpn_bbox/concat:0' shape=(None, None, 4) dtype=float32>", None, None, None, None)
(None, "<KerasTensor: shape=(8, None, 4) dtype=float32 (created by layer 'mrcnn_class_logits')>", None, None, None, None)
(None, "<KerasTensor: shape=(8, None, 4) dtype=float32 (created by layer 'mrcnn_class')>", None, None, None, None)
(None, "<KerasTensor: shape=(8, None, 4, 4) dtype=float32 (created by layer 'mrcnn_bbox')>", None, None, None, None)
(None, "<KerasTensor: shape=(8, None, 28, 28, 4) dtype=float32 (created by layer 'mrcnn_mask')>", None, None, None, None)
(None, "<KerasTensor: shape=(8, None, 4) dtype=float32 (created by layer 'ROI')>", None, None, None, None)
(None, "<KerasTensor: shape=(8, None, 4) dtype=float32 (created by layer

In [21]:
names = ['Alice', 'Bob', 'Charlie']
ages = [25, 32, 47]
scores = []

# Combine the elements from names, ages, and scores
combined = zip(names, ages, scores)

# Iterate over the combined iterable and print the tuples
for item in combined:
    print(item)

In [22]:
print(len(y_pred))
print(len(sample_weight))
print(len(t_losses))
print(len(t_loss_weights))
print(len(t_per_output_metrics))

14
14
14
14
14


In [10]:
import tensorflow as tf

# Define three input tensors
tensor1 = tf.constant([1, 2, 3])
tensor2 = tf.constant([4, 5, 6])
tensor3 = tf.constant([7, 8, 9])

# Perform element-wise addition using tf.add_n()
result = tf.add_n([tensor1, tensor2, tensor3])

# Print the result
print(result.numpy())

[12 15 18]


In [13]:
import tensorflow as tf

# Create a tensor with placeholder-like behavior initialized with 0
placeholder_tensor = tf.Variable(0, dtype=tf.float32, name='Placeholder')

In [14]:
placeholder_tensor

<tf.Variable 'Placeholder:0' shape=() dtype=float32, numpy=0.0>